In [2]:
!python --version

Python 3.10.16


In [51]:
GROQ_API_KEY = "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"


In [84]:
import os
from operator import itemgetter
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import AIMessage, SystemMessage, trim_messages
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_message_histories import ChatMessageHistory


import warnings
warnings.filterwarnings("ignore")



In [52]:
model = ChatGroq(model = "Gemma2-9b-It",groq_api_key = GROQ_API_KEY)

In [53]:
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000027E4AA90A60>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000027E4AA917E0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [54]:
messages = [
            SystemMessage(content = "Translate the following from english to french"),
            HumanMessage(content = "Hello how are you?")
]

'''SystemMessages Provides high-level instructions to the AI model.
Sets the behavior, tone, or context of the conversation.
Not visible to the user, only to the model.'''

'''When to Use SystemMessage vs. Just HumanMessage
If you want strict control over the model's behavior → Use SystemMessage.
If it's a one-time query and clarity isn't a concern → HumanMessage can work.'''

"When to Use SystemMessage vs. Just HumanMessage\nIf you want strict control over the model's behavior → Use SystemMessage.\nIf it's a one-time query and clarity isn't a concern → HumanMessage can work."

In [55]:
result = model.invoke(messages)

In [56]:
result

AIMessage(content="The most common translation is:\n\n**Bonjour, comment allez-vous ?** \n\nThis is a formal way of greeting someone. \n\nHere are some other options:\n\n* **Salut, ça va ?** (informal)\n* **Coucou, comment vas-tu ?** (very informal, used with friends)\n\n\nLet me know if you'd like more variations!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 82, 'prompt_tokens': 21, 'total_tokens': 103, 'completion_time': 0.149090909, 'prompt_time': 0.002212357, 'queue_time': 0.232886503, 'total_time': 0.151303266}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-7d02db40-1bbe-4b4e-aae9-1e68e74473fc-0', usage_metadata={'input_tokens': 21, 'output_tokens': 82, 'total_tokens': 103})

In [9]:
parser = StrOutputParser()
parser.invoke(result)

'Bonjour, comment allez-vous ? \n\n\nYou can also use the more informal:\n\n* Salut, ça va ? \n\n'

In [10]:
chain = model | parser

In [11]:
chain.invoke(messages)

'The most common way to say "Hello, how are you?" in French is:\n\n**Salut, comment vas-tu ?**  (Informal - used with friends and family) \n\n**Bonjour, comment allez-vous ?** (Formal - used with strangers or people you respect) \n\n\nLet me know if you\'d like to know other ways to say it!\n'

In [12]:
generic_template = "Translate the following into {language}:"

In [13]:
prompt = ChatPromptTemplate.from_messages([
          ("system",generic_template),
          ("user","{text}")    
])

In [14]:
res = prompt.invoke({"language":"french","text":"Hello"})
res.to_messages()

[SystemMessage(content='Translate the following into french:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [15]:
chain = prompt | model | parser

In [36]:
chain.invoke({"language": "french","text": "hello"})

'Bonjour  \n'

In [ ]:
#### other way

In [ ]:
generic_template = SystemMessage(content="Translate the following into {language}:")
user_template = HumanMessage(content="{text}")

prompt = ChatPromptTemplate.from_messages([
    generic_template,  # System message
    user_template      # User message
])

res = prompt.invoke({"language": "french", "text": "Hello"})
messages = res.to_messages()


In [ ]:
def get_dynamic_prompt(include_context: bool = False):
    """Dynamically creates a prompt template based on user preference."""
    messages = [
        ("system", "Translate the following into {language}:")
    ]
    
    if include_context:
        messages.append(("system", "Context: {context}"))
    
    messages.append(("user", "{text}"))

    return ChatPromptTemplate.from_messages(messages)

# Case 1: Basic Translation (No Context)
prompt1 = get_dynamic_prompt(include_context=False)
res1 = prompt1.invoke({"language": "French", "text": "Hello"})
print("Basic Translation Messages:", res1.to_messages())

# Case 2: Translation with Context
prompt2 = get_dynamic_prompt(include_context=True)
res2 = prompt2.invoke({"language": "French", "text": "Hello", "context": "Use a formal tone"})
print("Translation with Context Messages:", res2.to_messages())


In [37]:
#############################################################################################################################

In [38]:
# building chatbot with message history

In [39]:
model.invoke([HumanMessage(content = "Hi I am Alok and I am an AI engineer")])

AIMessage(content="Hi Alok! It's nice to meet you.  \n\nAs an AI engineer, what kind of projects are you working on these days?  \n\nI'm always interested in learning more about the work being done in the field of AI.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 20, 'total_tokens': 74, 'completion_time': 0.098181818, 'prompt_time': 0.001983657, 'queue_time': 0.234412493, 'total_time': 0.100165475}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-f96920c5-537e-4530-a54f-5d295225e4b3-0', usage_metadata={'input_tokens': 20, 'output_tokens': 54, 'total_tokens': 74})

In [43]:
model.invoke(
    [
        HumanMessage(content = "Hi I am Alok and I am an AI engineer"),
        AIMessage(content = "Hi Alok! It's nice to meet you. As an AI engineer, what kind of projects are you working on these days?   \
        I'm always interested in learning more about the work being done in the field of AI."),
        HumanMessage(content = "Hey whats my name and what do I do?")
    ]
)

AIMessage(content="You told me your name is Alok and that you are an AI engineer.  😊  \n\nIs there anything else you'd like to tell me about your work?  I'm curious to hear more!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 88, 'total_tokens': 135, 'completion_time': 0.085454545, 'prompt_time': 0.004135291, 'queue_time': 0.233859517, 'total_time': 0.089589836}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-3ae80d4a-6b73-4995-bf1e-6483e5c027f1-0', usage_metadata={'input_tokens': 88, 'output_tokens': 47, 'total_tokens': 135})

In [50]:
store = {}
def get_session_history(session_id:str)-> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model,get_session_history)    

In [53]:
config = {"configurable":{"session_id":"chat1"}}

In [55]:
response = with_message_history.invoke(
    [HumanMessage(content = "Hi I am Alok and I am an AI engineer")],
    config = config
)

In [56]:
response.content

"Hi Alok! It's great to meet you.  \n\nWhat kind of AI projects are you working on these days? Are you focused on a particular area like computer vision, natural language processing, or something else?  I'm always interested in learning about the cool things people are building with AI. 😊  \n\n"

In [58]:
with_message_history.invoke(
    [HumanMessage(content = "What is my name?")],
    config = config
)

AIMessage(content='Your name is Alok. 😊  I remember that you introduced yourself at the beginning of our conversation.  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 185, 'total_tokens': 210, 'completion_time': 0.045454545, 'prompt_time': 0.007582296, 'queue_time': 0.233232709, 'total_time': 0.053036841}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-f29b3b42-7b98-48b2-92d9-bca1e9f3f6ce-0', usage_metadata={'input_tokens': 185, 'output_tokens': 25, 'total_tokens': 210})

In [60]:
config1 = {"configurable":{"session_id":"chat2"}}

with_message_history.invoke(
    [HumanMessage(content = "What is my name?")],
    config = config1
)

AIMessage(content="As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to know! 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 14, 'total_tokens': 53, 'completion_time': 0.070909091, 'prompt_time': 0.001927946, 'queue_time': 0.23523087499999998, 'total_time': 0.072837037}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-60f9d9bd-a4f2-451f-88ac-4ce33af5a150-0', usage_metadata={'input_tokens': 14, 'output_tokens': 39, 'total_tokens': 53})

In [62]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","you are hlpful assistant. Answer all the questions to the nest of your ability"),
        MessagesPlaceholder(variable_name = "messages")
    ]
)

chain = prompt|model

In [63]:
chain.invoke({"messages":[HumanMessage(content="Hi my name is Alok")]})

AIMessage(content='Hi Alok! Nice to meet you. \n\nHow can I be helpful today?  \n\nAsk me anything! 😊  \n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 32, 'total_tokens': 62, 'completion_time': 0.054545455, 'prompt_time': 0.002364067, 'queue_time': 0.234520408, 'total_time': 0.056909522}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-1f9d6ce8-57bd-4d8b-a239-72903cc6d3ca-0', usage_metadata={'input_tokens': 32, 'output_tokens': 30, 'total_tokens': 62})

In [64]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history)
config = {"configurable":{"session_id":"chat3"}}

In [65]:
response = with_message_history.invoke(
    [HumanMessage(content="My name is Alok")],
    config = config
)

response

AIMessage(content="Hello Alok!\n\nIt's nice to meet you. I'm happy to be your helpful assistant. \n\nAsk me anything! I'll do my best to answer your questions thoroughly and helpfully.  \n\nWhat can I do for you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 31, 'total_tokens': 91, 'completion_time': 0.109090909, 'prompt_time': 0.002423036, 'queue_time': 0.234762249, 'total_time': 0.111513945}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-a8f32647-82ba-480d-8679-618fdc2706cb-0', usage_metadata={'input_tokens': 31, 'output_tokens': 60, 'total_tokens': 91})

In [68]:
response2 = with_message_history.invoke(
    [HumanMessage(content="whats is My name?")],
    config = config
)

response2.content

"Your name is Alok.  \n\nI'm here to help, but I'm starting to think you might be testing me! 😄  Is there anything else you'd like to know or talk about? \n\n\n\n"

In [ ]:
session1_config = {"configurable": {"session_id": "session_1"}}
session2_config = {"configurable": {"session_id": "session_2"}}

response1 = with_message_history.invoke(
    [HumanMessage(content="Hi, my name is Alok, and I am an AI engineer.")],
    config=session1_config
)
print("Session 1 - Response 1:", response1.content)



response2 = with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=session1_config
)
print("Session 1 - Response 2:", response2.content)



response3 = with_message_history.invoke(
    [HumanMessage(content="Do you remember my name?")],
    config=session2_config
)
print("Session 2 - Response 1:", response3.content)


In [70]:
##################### little complexity addition

In [71]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","you are hlpful assistant. Answer all the questions to the nest of your ability in {language}"),
        MessagesPlaceholder(variable_name = "messages")
    ]
)

chain = prompt|model

In [72]:
response = chain.invoke({"messages":[HumanMessage(content="Hi my name is Alok")],"language":"Hindi"})

response.content

'नमस्ते अलोक!  😊\n\nमैं आपकी मदद करने के लिए तैयार हूँ। आपके कोई सवाल हैं? \n'

In [89]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

config = {"configurable":{"session_id":"chat5"}}

response = with_message_history.invoke(
    {"messages":[HumanMessage(content="Hi I am Alok")],
     "language":"Hindi"},
    config = config
)

response.content

'नमस्ते अलोक! \n\nमैं आपकी मदद करने के लिए यहाँ हूँ। आपके कोई सवाल हैं तो पूछने में संकोच न करें। मैं अपनी पूरी क्षमता के साथ उत्तर देने की कोशिश करूँगा। 😊\n'

In [93]:
trimmer = trim_messages(
    max_tokens = 70,
    strategy = "last",
    token_counter = model,
    include_system = True,
    allow_partial = False,
    start_on = "human"
)

messages = [
    SystemMessage(content="you are a good assistant"),
    HumanMessage(content = "hi I am bob"),
    AIMessage(content = "Hi"),

    HumanMessage(content = "I like vanilla ice cream"),
    AIMessage(content = "nice"),

    HumanMessage(content = "what is 2+2"),
    AIMessage(content = "4"),

    HumanMessage(content = "thanks!"),
    AIMessage(content = "no problem"),

    HumanMessage(content = "havig fun?"),
    AIMessage(content = "yes!"),
]

In [95]:
trimmer.invoke(messages)

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

C:\Users\Hp\anaconda3\envs\py310_env\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Hp\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

[SystemMessage(content='you are a good assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi I am bob', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is 2+2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='havig fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [101]:
chain = (RunnablePassthrough.assign(messages = itemgetter("messages")|trimmer) | prompt | model )

chain.invoke(
    {
    "messages":messages + [HumanMessage(content="what ice cream do I like?")],
     "language":"English"
}
)

AIMessage(content='You said you like vanilla ice cream! 😄  \n\n\n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 119, 'total_tokens': 132, 'completion_time': 0.023636364, 'prompt_time': 0.005438142, 'queue_time': 0.23287908599999999, 'total_time': 0.029074506}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-06149dcf-bb4d-427f-ac5e-42418b29a3d9-0', usage_metadata={'input_tokens': 119, 'output_tokens': 13, 'total_tokens': 132})

In [102]:
chain.invoke(
    {
    "messages":messages + [HumanMessage(content="what is 9+13")],
     "language":"English"
}
)

AIMessage(content='22 \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 119, 'total_tokens': 125, 'completion_time': 0.010909091, 'prompt_time': 0.005391692, 'queue_time': 0.232858677, 'total_time': 0.016300783}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-2b308595-de21-487e-bd01-240359fa28fb-0', usage_metadata={'input_tokens': 119, 'output_tokens': 6, 'total_tokens': 125})

In [104]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config = {"configurable":{"session_id":"chat6"}}

In [105]:
response = with_message_history.invoke(
    {
    "messages":messages + [HumanMessage(content="what is 9+13")],
     "language":"English"
},
    config = config
)

response.content

'22 🍦  '

In [5]:
######################### RAG system implementation ###################################

In [85]:
from langchain_core.documents import Document
from langchain_chroma import Chroma
from sentence_transformers import SentenceTransformer
from langchain_chroma import Chroma
# from langchain_huggingface import HuggingFaceEmbeddings
from typing import List
from langchain_core.runnables import RunnableLambda

In [86]:
documents = [
    Document(
        page_content = "Dogs are great companions, known for their loyalty and friendliness",
        metadata = {"source": "mammel-pets-doc"},
    ),
    Document(
        page_content = "cats are independent pets that often enjoy their own space.",
        metadata = {"source": "mammel-pets-doc"},
    ),
    Document(
        page_content = "Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata = {"source": "fish-pets-doc"},
    ),
    Document(
        page_content = "Parrots are intelligent birds capable of mimicking human beings",
        metadata = {"source": "bird-pets-doc"},
    ),
    Document(
        page_content = "Rabbits are social animals that need plenty of space to hop around",
        metadata = {"source": "mammel-pets-doc"},
    ),
]

In [87]:
documents

[Document(metadata={'source': 'mammel-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness'),
 Document(metadata={'source': 'mammel-pets-doc'}, page_content='cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human beings'),
 Document(metadata={'source': 'mammel-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around')]

In [88]:
llm = ChatGroq(groq_api_key = GROQ_API_KEY, model = 'Llama3-8b-8192')

In [89]:
vectorstore = Chroma.from_documents(documents,embedding_model)

In [90]:
vectorstore

In [91]:
# async query

await vectorstore.asimilarity_search("cat")

[Document(id='13eb34c2-06ce-49c1-a080-57023846e1c9', metadata={'source': 'mammel-pets-doc'}, page_content='cats are independent pets that often enjoy their own space.'),
 Document(id='02380838-ce7c-4168-99de-646cbbe71a2e', metadata={'source': 'mammel-pets-doc'}, page_content='cats are independent pets that often enjoy their own space.'),
 Document(id='1a082c80-0997-4c07-ba83-e1beff50e007', metadata={'source': 'mammel-pets-doc'}, page_content='cats are independent pets that often enjoy their own space.'),
 Document(id='fe4c6528-0240-4565-97a1-5e8e0d9d123e', metadata={'source': 'mammel-pets-doc'}, page_content='cats are independent pets that often enjoy their own space.')]

In [92]:
await  vectorstore.asimilarity_search_with_score("cat")

[(Document(id='13eb34c2-06ce-49c1-a080-57023846e1c9', metadata={'source': 'mammel-pets-doc'}, page_content='cats are independent pets that often enjoy their own space.'),
  0.9351054430007935),
 (Document(id='02380838-ce7c-4168-99de-646cbbe71a2e', metadata={'source': 'mammel-pets-doc'}, page_content='cats are independent pets that often enjoy their own space.'),
  0.9351054430007935),
 (Document(id='1a082c80-0997-4c07-ba83-e1beff50e007', metadata={'source': 'mammel-pets-doc'}, page_content='cats are independent pets that often enjoy their own space.'),
  0.9351054430007935),
 (Document(id='fe4c6528-0240-4565-97a1-5e8e0d9d123e', metadata={'source': 'mammel-pets-doc'}, page_content='cats are independent pets that often enjoy their own space.'),
  0.9351054430007935)]

In [93]:
# retrievers 

In [94]:
retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)

In [95]:
retriever.batch(["cat","dog"])

[[Document(id='1a082c80-0997-4c07-ba83-e1beff50e007', metadata={'source': 'mammel-pets-doc'}, page_content='cats are independent pets that often enjoy their own space.')],
 [Document(id='ec0c59fa-322c-4423-a5e6-a53c84e973df', metadata={'source': 'mammel-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness')]]

In [96]:
vectorstore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k":1}
)
retriever.batch(["cat","dog"])

[[Document(id='1a082c80-0997-4c07-ba83-e1beff50e007', metadata={'source': 'mammel-pets-doc'}, page_content='cats are independent pets that often enjoy their own space.')],
 [Document(id='ec0c59fa-322c-4423-a5e6-a53c84e973df', metadata={'source': 'mammel-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness')]]

In [97]:
message = """
Answer this question using the provided context only
{question}
Context:
{context}
"""

In [98]:
prompt = ChatPromptTemplate.from_messages([("human",message)])

rag_chain = {"context":retriever,"question":RunnablePassthrough()}|prompt|llm

response = rag_chain.invoke("tell me about dogs")

print(response.content)

According to the provided context, dogs are great companions, known for their loyalty and friendliness.
